**Workflow for finding transfer volumes:**

- Given target and stock concentrations, calculate volumes for transfer

- If all volumes are > 1uL we don't need dilutions and can proceed to transfers

- Otherwise, find those that are within [0.1, 1) and dilute 10x; for [0.01, 0.1) dilute 100x; for [0.001, 0.01) dilute 1000x

- Recalculate required volumes for transfers

- Transfer

### Derivation

Molar concentration [mol/L] is given as:

$$c=\frac{n}{V}$$

Volume of stock component $i$ with concentration $c_{s_i}$ needed for a target concentration $c_{t_i}$ and total volume $V_T$ is

$$
V_{s_i}=\frac{n_{s_i}}{c_{s_i}}=\frac{c_{t_i}V_T}{c_{s_i}}, \; \forall i
\label{volume_component} \tag{1}
$$

Conditions for checking the algoritm implementation and inputs: 

**Condition 1.** 
$$V_T, V_w, V_{s_i}, c_{t_i}, c_{s_i}\geq0, \forall i \tag{C1} \label{cond1}$$

**Condition 2.** 
$$ c_{t_i}\leq c_{s_i}, \forall i \tag{C2} \label{cond2}$$


Total volume is given as the sum of volumes for all components and water, i.e.

\begin{equation}
V_T=\sum_iV_{s_i}+V_w
\tag{2}
\label{total_volume}
\end{equation}

From $\eqref{volume_component}$, $\eqref{total_volume}$ and $\eqref{cond1}$:

$$V_T=V_T\sum_i\frac{c_{t_i}}{c_{s_i}}+V_w \geq V_T\sum_i\frac{c_{t_i}}{c_{s_i}}$$

which implies

**Condition 3.** 
$$ \sum_i\frac{c_{t_i}}{c_{s_i}}\leq 1\tag{C3} \label{cond3}$$

$\eqref{volume_component}$ and $\eqref{total_volume}$ give rise to the following system of linear equations to solve for volumes $V_{s_i}, \forall i$

\begin{cases}
\left(\frac{c_{t_i}}{c_{s_i}}-1\right)V_{s_i}+ \frac{c_{t_i}}{c_{s_i}} \left( \sum_{j\neq i}V_{s_j} + V_w\right)=0,  \forall i \\
\sum_jV_{s_j}+V_w = V_T
\end{cases}





This results in the system 

$$Ax=b,$$

where

$$A=
\begin{bmatrix}
\frac{c_{t_1}}{c_{s_1}}-1 & \frac{c_{t_1}}{c_{s_1}} & \dots & \frac{c_{t_1}}{c_{s_1}} & \frac{c_{t_1}}{c_{s_1}}\\
\frac{c_{t_2}}{c_{s_2}} & \frac{c_{t_2}}{c_{s_2}}-1 & \dots & \frac{c_{t_2}}{c_{s_2}} & \frac{c_{t_2}}{c_{s_2}}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
\frac{c_{t_N}}{c_{s_N}} & \frac{c_{t_N}}{c_{s_N}} & \dots & \frac{c_{t_N}}{c_{s_N}}-1 & \frac{c_{t_N}}{c_{s_N}}\\
 1 & 1 & \dots & 1 & 1
\end{bmatrix}, \;
x=
\begin{bmatrix}
V_{s_1}\\
V_{s_2}\\
\vdots \\
V_{s_N}\\
V_{w}
\end{bmatrix}, \;
b=
\begin{bmatrix}
0\\
0\\
\vdots \\
0\\
V_T
\end{bmatrix}
$$

In [8]:
import numpy as np

In [1]:
cd ../

/Users/tradivojevic/Documents/Code/Python/OT/combinatorial_media


In [2]:
from comb_media.core import find_volumes

/Users/tradivojevic/.opentrons/deck_calibration.json not found. Loading defaults
/Users/tradivojevic/.opentrons/robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


Test data for the algorithm. Final volumes (after dilutions) should be 50uL for each component.

In [3]:
stock_conc_file = 'data/stock_concentrations_DBTL0.csv'
target_conc_file = 'data/target_concentrations_DBTL0.csv'
well_volume = 1000

In [4]:
volumes, df = find_volumes(stock_conc_file, target_conc_file, well_volume)

In [5]:
volumes

array([4.0000e+01, 4.0000e+00, 4.0000e-02, 2.0000e+01, 5.0000e+01,
       5.0000e+01, 5.0000e-01, 5.0000e+01, 5.0000e-02, 5.0000e+00,
       5.0000e+01, 5.0000e-02, 5.0000e-02, 5.0000e-02, 5.0000e-02,
       5.0000e-02, 7.3016e+02])

In [6]:
df

,Master Solution Concentration [M],Volumes[uL]
Media Component,,
MOPS,1.0000,40.00
Tricine,1.0000,4.00
H3BO3,0.1000,0.04
Glucose,1.0000,20.00
K2SO4,0.0058,50.00
K2HPO4,0.0264,50.00
FeSO4,0.0200,0.50
NH4Cl,0.1904,50.00
CaCl2,0.0100,0.05


In [9]:
# Already calculated dilutions for this case:
df['Dilution factor(DF)']= np.array([1.25, 12.5, 1250, 2.5, 1, 1, 100, 1, 1000, 10, 1, 1000,1000,1000,1000,1000])

Final volumes for all components should be 50nL

In [10]:
df['DF*Vol[uL]'] = df['Volumes[uL]']*df['Dilution factor(DF)']

In [11]:
df

,Master Solution Concentration [M],Volumes[uL],Dilution factor(DF),DF*Vol[uL]
Media Component,,,,
MOPS,1.0000,40.00,1.25,50.0
Tricine,1.0000,4.00,12.50,50.0
H3BO3,0.1000,0.04,1250.00,50.0
Glucose,1.0000,20.00,2.50,50.0
K2SO4,0.0058,50.00,1.00,50.0
K2HPO4,0.0264,50.00,1.00,50.0
FeSO4,0.0200,0.50,100.00,50.0
NH4Cl,0.1904,50.00,1.00,50.0
CaCl2,0.0100,0.05,1000.00,50.0


## Including culture ratio into account

Given culture ratio $c_\text{ratio}$ (e.g. 100x), culture volume is defined as

$$V_c=\frac{V_T}{c_\text{ratio}}$$

Volume of stock component $i$ with concentration $c_{s_i}$ needed for a target concentration $c_{t_i}$ and total volume $V_T$ is

$$
V_{s_i}=\frac{n_{s_i}}{c_{s_i}}=\frac{c_{t_i}V_T}{c_{s_i}}, \; \forall i
\label{volume_component} \tag{1}
$$

Conditions for checking the algoritm implementation and inputs: 

**Condition 1.** 
$$V_T, V_w, V_{s_i}, V_c, c_{t_i}, c_{s_i}\geq0, \forall i \tag{C1} \label{cond1}$$

**Condition 2.** 
$$ c_{t_i}\leq c_{s_i}, \forall i \tag{C2} \label{cond2}$$


Total volume is given as the sum of volumes for all components, culture and water, i.e.

\begin{equation}
V_T=\sum_iV_{s_i}+V_w +V_c
\tag{2}
\label{total_volume}
\end{equation}

From $\eqref{volume_component}$, $\eqref{total_volume}$ and $\eqref{cond1}$:

$$V_T=V_T\sum_i\frac{c_{t_i}}{c_{s_i}}+V_w+V_c \geq V_T\sum_i\frac{c_{t_i}}{c_{s_i}}$$

which implies

**Condition 3.** 
$$ \sum_i\frac{c_{t_i}}{c_{s_i}}\leq 1\tag{C3} \label{cond3}$$

$\eqref{volume_component}$ and $\eqref{total_volume}$ give rise to the following system of linear equations to solve for volumes $V_{s_i}, \forall i$

\begin{cases}
\left(\frac{c_{t_i}}{c_{s_i}}-1\right)V_{s_i}+ \frac{c_{t_i}}{c_{s_i}} \left( \sum_{j\neq i}V_{s_j} + V_w+V_c\right)=0,  \forall i \\
\sum_jV_{s_j}+V_w+V_c = V_T
\end{cases}





This results in the system 

$$Ax=b,$$

where

$$A=
\begin{bmatrix}
\frac{c_{t_1}}{c_{s_1}}-1 & \frac{c_{t_1}}{c_{s_1}} & \dots & \frac{c_{t_1}}{c_{s_1}} & \frac{c_{t_1}}{c_{s_1}}\\
\frac{c_{t_2}}{c_{s_2}} & \frac{c_{t_2}}{c_{s_2}}-1 & \dots & \frac{c_{t_2}}{c_{s_2}} & \frac{c_{t_2}}{c_{s_2}}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
\frac{c_{t_N}}{c_{s_N}} & \frac{c_{t_N}}{c_{s_N}} & \dots & \frac{c_{t_N}}{c_{s_N}}-1 & \frac{c_{t_N}}{c_{s_N}}\\
 1 & 1 & \dots & 1 & 1
\end{bmatrix}, \;
x=
\begin{bmatrix}
V_{s_1}\\
V_{s_2}\\
\vdots \\
V_{s_N}\\
V_{w}
\end{bmatrix}, \;
b=-V_c
\begin{bmatrix}
\frac{c_{t_1}}{c_{s_1}}\\
\frac{c_{t_2}}{c_{s_2}}\\
\vdots \\
\frac{c_{t_N}}{c_{s_N}}\\
1-\frac{V_T}{V_c}
\end{bmatrix}
$$